# Corridor experiments

In this notebook, we look at matchings with corridor navground robots. 

First, let us import a few important modules for this task.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist
import matplotlib as mpl

import os

from navground import core, sim

import perdiver.perdiver as perdiver
from perdiver.distances import *

plots_dir = os.path.join("plots", "matchings")
experiment_dir = "experiments"
os.makedirs(plots_dir, exist_ok=True)
os.makedirs("experiments", exist_ok=True)

Next, let us execute the Navground corridor experiment.

In [ ]:
length = 8.0
num_steps = 500
width=1.0
num_agents = 31 # 38
behaviour_list = ["ORCA", "HL", "HRVO", "Dummy"]
for behaviour in behaviour_list:
    path = os.path.join(experiment_dir, f"matchings_{behaviour}.h5")
    yaml = f"""
    steps: {num_steps}
    time_step: 0.1
    record_pose: true
    record_twist: true
    runs: 1
    scenario:
      type: Corridor
      length: {length}
      width: {width} 
      groups:
        -
          type: thymio
          number: {num_agents}
          radius: 0.08
          control_period: 0.1
          speed_tolerance: 0.02
          kinematics:
            type: 2WDiff
            wheel_axis: 0.094
            max_speed: 0.166
          behavior:
            type: {behaviour}
            optimal_speed: 0.12
            horizon: 5.0
            safety_margin: 0.03
          state_estimation:
            type: Bounded
            range: 5.0
    """
    experiment = sim.load_experiment(yaml)
    experiment.run(keep=False, data_path=path)
    del experiment

Reload both experiments and save runs into a dictionary.

In [ ]:
runs = {}
# Reload simulations
for behaviour in behaviour_list:
    path = os.path.join(experiment_dir, f"matchings_{behaviour}.h5")
    recorded_experiment = sim.RecordedExperiment(path)
    runs[behaviour] = recorded_experiment.runs

Let us display this simulation in a video

In [ ]:
from navground.sim.ui import WebUI
from navground.sim.notebook import notebook_view
from navground.sim.replay import RealTimeReplay

web_ui = WebUI(host='127.0.0.1', max_rate=-1)
await web_ui.prepare()

First, let us visualise the HL experiment.

In [ ]:
notebook_view(width=300)

In [ ]:
rt_sim = RealTimeReplay(run=runs["HL"][0], factor=10, web_ui=web_ui)
await rt_sim.run()

Now, we visualise the ORCA experiment.

In [ ]:
rt_sim = RealTimeReplay(run=runs["ORCA"][0], factor=10, web_ui=web_ui)
await rt_sim.run()

Also HRVO and Dummy

In [ ]:
rt_sim = RealTimeReplay(run=runs["HRVO"][0], factor=10, web_ui=web_ui)
await rt_sim.run()

In [ ]:
rt_sim = RealTimeReplay(run=runs["Dummy"][0], factor=10, web_ui=web_ui)
await rt_sim.run()

Both simulations are very different. In partiuclar, we observe mainly two dynamics, either the robots end up going in straight trajectories or they get stuck. Both examples below where produced with the same variables.

### Pairwise Matchings

We are now going to compute the induced matchings and their associated diagrams.

Also, we set up the variable "weight" and the timestep shift for our experiments.

In [ ]:
weight = 1 # This is the weight for velocities
shift_time = 20 # Shift between timesteps
steps_list = list(range(20, 120, 10)) # Starting timesteps that we consider

Now, we are going to start by considering two timesteps and their associated divergence diagrams. Notice that these do not change much.

In [ ]:
start_step = 20
gs_kw = dict(width_ratios=[5, 2, 2], height_ratios=[1,1])
fig, axd = plt.subplot_mosaic([['points_HL', 'Diag_HL', 'Diag_ORCA'],
                               ['points_ORCA', 'Diag_HL', 'Diag_ORCA']],
                              gridspec_kw=gs_kw, figsize=(16, 4),
                              layout="constrained")
for behaviour in ["HL", "ORCA"]:
    ps = np.array(runs[behaviour][0].poses)[:,:]
    twists = np.array(runs[behaviour][0].twists)[:,:]
    X = ps[start_step]
    Y = ps[start_step + shift_time]
    vel_X = twists[start_step]
    vel_Y = twists[start_step + shift_time]
    X_len = X.shape[0]-1
    # Plot two timesteps
    # fig, ax = plt.subplots(figsize=(8, 1))
    ax = axd[f"points_{behaviour}"]
    perdiver.plot_two_timesteps_with_velocities(X[:,:2], Y[:,:2], vel_X[:,:2], vel_Y[:,:2], ax, arrow_width=0.03)
    ax.set_title(f"points_{behaviour}", fontsize=20)
    # Plot matching diagram
    ax = axd[f"Diag_{behaviour}"]
    Dist_X, Dist_Y, Dist_Z = perdiver.compute_distance_matrices_timesteps_corridor(X, Y, vel_X, vel_Y, weight, length)
    match_diagram = perdiver.get_matching_diagram(Dist_X, Dist_Y)
    perdiver.plot_matching_diagram(match_diagram, ax, color="blue")
    ax.set_title(behaviour, fontsize=20)

perdiver.same_diagram_scale(axd["Diag_HL"], axd["Diag_ORCA"])
plt.savefig(os.path.join(plots_dir, f"two_timesteps_points_matching_diags_ORCA_HL.png"))

Last, we compute the persistence matching diagram across the step list. There seems to be an equilibrium.

In [ ]:
fig, ax = plt.subplots(ncols=len(behaviour_list), figsize=(5*len(behaviour_list),4))
diagrams_behaviour = {}
for j, behaviour in enumerate(behaviour_list):
    ps = np.array(runs[behaviour][0].poses)
    twists = np.array(runs[behaviour][0].twists)
    diagrams_list = []
    for idx, start_step in enumerate(steps_list):
        X = ps[start_step]
        Y = ps[start_step + shift_time]
        vel_X = twists[start_step]
        vel_Y = twists[start_step+shift_time]
        Dist_X, Dist_Y, Dist_Z = perdiver.compute_distance_matrices_timesteps_corridor(X, Y, vel_X, vel_Y, weight, length)
        match_diagram = perdiver.get_matching_diagram(Dist_X, Dist_Y)
        perdiver.plot_matching_diagram(match_diagram, ax[j], color=mpl.colormaps["GnBu"](idx/len(steps_list)))
        diagrams_list.append(match_diagram)
    # end for 
    diagrams_behaviour[f"{behaviour}"] = diagrams_list
    norm = mpl.colors.Normalize(vmin=steps_list[0], vmax=steps_list[-1])
    cmap = mpl.colormaps["GnBu"]
    mappable = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
    plt.colorbar(mappable=mappable, ax=ax[j])
    ax[j].set_title(f"Evolution matching diagram {behaviour}")
# end for
perdiver.same_diagram_scale(ax[0], ax[1])
plt.savefig(os.path.join(plots_dir, f"evolution_matching_ORCA_HL.png"))

### Vectorisation of matching diagrams

Use persistence images for this.

In [ ]:
from gudhi import representations

In [ ]:
npixels = 15
perim = representations.PersistenceImage(resolution=[npixels, npixels], bandwidth=0.1, im_range=[0,3,-3,3])
all_diagrams = []
for behaviour in behaviour_list:
    all_diagrams += diagrams_behaviour[behaviour]
perim.fit(all_diagrams)
perim_arr_dict = {}
for behaviour in behaviour_list:
    perim_arr_dict[behaviour] = perim.transform(diagrams_behaviour[behaviour])

Visualise persistence images.

In [ ]:
fig, ax = plt.subplots(ncols=len(behaviour_list), figsize=(3.5*len(behaviour_list),3), layout="constrained")
for idx, behaviour in enumerate(behaviour_list):
    # mean_perim = perim_arr_dict[behaviour].mean(axis=0).reshape(15,15)
    # mean_perim = (mean_perim / mean_perim.max(axis=0))
    # ax[idx].imshow(mean_perim)
    ax[idx].imshow(perim_arr_dict[behaviour].mean(axis=0).reshape(15,15))
    ax[idx].set_title(behaviour, fontsize=20)
# end for 
# plt.tight_layout()
plt.savefig(os.path.join(plots_dir, f"image_persistence_ORCA_HL.png"))

In [ ]:
def timestep_to_idx(steps_list, time):
    return np.argmax((np.array(steps_list) - 20)>=0)

Measure differeces across time.

### Divergence vectors across time

Next, we compute and print the divergence array across a few simulation steps.

In [ ]:
divergence_dict = {}
for behaviour in behaviour_list:
    ps = np.array(runs[behaviour][0].poses)
    twists = np.array(runs[behaviour][0].twists)
    divergence_list = []
    Z_barcodes_list = []
    for start_step in steps_list:
        X = ps[start_step]
        Y = ps[start_step + shift_time]
        vel_X = twists[start_step]
        vel_Y = twists[start_step + shift_time]
        Dist_X, Dist_Y, Dist_Z = perdiver.compute_distance_matrices_timesteps_corridor(X, Y, vel_X, vel_Y, weight, length)
        divergence_vector = np.sort(perdiver.compute_divergence_vector(Dist_X, Dist_Y))
        divergence_list.append(divergence_vector)

    divergence_dict[behaviour] = np.array(divergence_list).transpose()

vmax = np.max([np.max(divergence_dict[behaviour]) for behaviour in divergence_dict.keys()])
vmin = np.min([np.min(divergence_dict[behaviour]) for behaviour in divergence_dict.keys()])
for behaviour in divergence_dict.keys():
    divergence_arr = divergence_dict[behaviour]
    ## Save figure 
    fig, ax = plt.subplots(figsize=(10,2))
    mapable = ax.imshow(divergence_arr, aspect="auto", vmax=vmax, vmin=vmin, extent=(steps_list[0], steps_list[-1], 0, X.shape[0]))
    ax.set_title(f"Divergence {behaviour}")
    plt.colorbar(mapable)
    plt.tight_layout()
    plt.savefig(os.path.join(plots_dir, f"Divergence_vector_evolution_{behaviour}.png"))